In [ ]:
import math
import numpy as np

from common import *
from mcs_shared import load_day, load_flight, load_model, load_topo, \
    ACCUMULATION_FLIGHTS, xr

In [ ]:
%load_ext autoreload
%autoreload 2

## Snow Depth Map resolution

In [ ]:
RESOLUTION = 10 # meters

## Get common area across all flights

In [ ]:
pd.options.plotting.backend = 'matplotlib'

In [ ]:
def common_area(flights):
    als_depth = []
    masks = []

    for flight in flights:
        als, mask = load_flight(
            flight, RESOLUTION, masked=False
        )
        
        als_depth.append(als)
        masks.append(mask.values)

    return als_depth, np.logical_and.reduce(np.stack(masks))
    
def area_factors(als_depth, common_area):
    als_area = []
    als_factors = []
    als_depth_masked = []
    
    for als in als_depth:
        als.coords['mask'] = (('y', 'x'), common_area)
        # Reduce to common area
        als = als.where(als.mask, drop=True)

        als_depth_masked.append(als)
                
        mean = float(als.snowdepth.mean().values)
        als = (als.snowdepth / mean)

        als_area.append(als)

        als_f = als.values.flatten()
        als_f = als_f[~np.isnan(als_f)]

        als_factors.append(als_f)

    return als_factors, als_depth_masked, als_area

In [ ]:
als_depth, joined_mask = common_area(ACCUMULATION_FLIGHTS)
factors, depths, areas = area_factors(
    als_depth, joined_mask
)

## Depths

In [ ]:
COLORS = [
    'royalblue', 'peru', 'teal', 'lightcoral', 'blueviolet', 'seagreen'
]

In [ ]:
fig, ax = plt.subplots(dpi=300, figsize=(10,4))

for idx, date in enumerate(ACCUMULATION_FLIGHTS):
    pd.DataFrame(
        depths[idx].snowdepth.values.flatten(), 
        columns=[pd.to_datetime(date).strftime('%Y-%m-%d')]
    ).plot(
        kind='density', ax=ax, lw=1, color=COLORS[idx], alpha=0.8
    )

ax.set_xlim(0,4.5)
ax.set_xlabel('Snow Depth (m)')
ax.set_ylim(-0.05, 1.6);

## Factors

In [ ]:
fig, ax = plt.subplots(dpi=300, figsize=(10,4))

for idx, date in enumerate(ACCUMULATION_FLIGHTS):
    pd.DataFrame(
        factors[idx], 
        columns=[pd.to_datetime(date).strftime('%Y-%m-%d')]
    ).plot(
        kind='density', ax=ax, lw=1, color=COLORS[idx], alpha=0.8
    )

ax.set_xlim(0,2)
ax.set_xlabel('Factors')
ax.set_ylim(-0.05, 2);

### Elevation

In [ ]:
topo = load_topo(joined_mask)

In [ ]:
def reduce_nan_xarray(variable):
    flattened = variable.values.flatten()
    return flattened[~np.isnan(flattened)]

In [ ]:
elevation = topo.dem.values.flatten()
elevation = elevation[~np.isnan(elevation)]

In [ ]:
coeff = []
intercept = []

for factor in factors:
    c, i = np.polyfit(factor, elevation, deg=1)
    coeff.append(c)
    intercept.append(i)

In [ ]:
fig, axes = plt.subplots(
    ncols=len(ACCUMULATION_FLIGHTS) + 1, 
    dpi=300, figsize=(10,5), sharex=True, sharey=True
)
x_val = np.linspace(0, 2, num=40)

for index, date in enumerate(ACCUMULATION_FLIGHTS):
    date_label = pd.to_datetime(date).strftime('%Y-%m-%d')
    cmap = axes[index].scatter(
        factors[index], elevation, 
        c=reduce_nan_xarray(depths[index].snowdepth), 
        s=0.8, cmap='cividis', clim=(0, 4)
    )
    axes[index].set_title(label=date_label)
    
    # Last panel regression line
    axes[-1].plot(
        x_val, intercept[index] + coeff[index] * x_val, 
        label=date_label,
        lw=1.5, alpha=0.8, ls='--'
    )
    axes[-1].legend(
        ncol=2, 
        loc='lower center',
        bbox_to_anchor=(-0.2, -0.25), 
        prop={'size': 8}
    )

axes[0].set_ylim(1580, 2500)
axes[0].set_xlim(0, 3)
axes[0].set_ylabel('Elevation (m)')
axes[3].set_xlabel('Factors')

cax = fig.add_axes([0.12, -0.015, 0.5, 0.03])
fig.colorbar(
    cmap, cax=cax, orientation='horizontal', 
    fraction=.05, label='Snow Depth (m)'
);

## Holoviews

In [ ]:
from holoviews import opts, Cycle

In [ ]:
use_hvplot()

### Density plots 

In [ ]:
def plot_flights(flights, factors, depths, title):
    return hv.Layout([
            hv.NdOverlay(
                { flights[n]: hv.Distribution(factors[n]) for n in range(0, len(flights)) }
            ).opts(
                opts.Distribution(filled=False, line_color=Cycle())
            ).opts(**HV_PLOT_OPTS, title='Factors', xlim=(0, 2), xlabel='Factors'),
            hv.NdOverlay(
                { 
                    flights[n]: hv.Distribution(depths[n].snowdepth.values.flatten()).opts(tools=['hover'])
                    for n in range(0, len(flights)) 
                }
            ).opts(
                opts.Distribution(filled=False, line_color=Cycle())
            ).opts(**HV_PLOT_OPTS, title='Depths', xlim=(0, 4), xlabel='Depths')
        ]).opts(shared_axes=False, title=title).cols(1)

In [ ]:
plot_flights(ACCUMULATION_FLIGHTS, factors, depths, "Accumulation Flights")

## Gamma

Plot CDF for depths 

In [ ]:
from scipy import stats
import pprint
np.set_printoptions(precision=3, suppress=True)

In [ ]:
def fit_norm(flight, idx, x):
    param = stats.norm.fit(flight)
    return stats.norm.cdf(x, *param)

In [ ]:
def fit_gamma(flight, idx, x):
    param = stats.gamma.fit(flight)
    return stats.gamma.cdf(x, *param)    

In [ ]:
fig, ax = plt.subplots(dpi=300)

for idx, flight in enumerate(factors):
    ecdf_f = stats.ecdf(flight)
    ecdf_f.cdf.plot(ax, label=f'{ACCUMULATION_FLIGHTS[idx]}')
    # print(peak_als_depth[idx])
    # pprint.pp(ecdf_f.cdf.quantiles)

ax.set_title("ECDF")
ax.legend()
plt.show()